# Module 9 : Introduction to Apache Spark

## Assignmgnet 4 : Boa Kim( BK2953)

### 1. Set up the environment

Details¶

1. Find all companies with names that contain exactly three words  (e.g. "Fox Interactive Media") 
- Print the count of such companies 

2. Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame
- Find all companies located in the state of New Jersey:
- Print the count of such companies

3. Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame
- Add a "TechDomain" column to the DataFrame where the row entries are set to 1 if the "domain" field contains "github.io", and 0 otherwise:
- Use show() to display only the name, location (city, region, country_code) and TechDomain column for companies where the TechDomain  field is marked as 1.
4. Find all companies whose names contain at least one repeating consecutive letter (e.g., "Google", "Twitter", "Massa Inc.") using a Spark UDF function.
-Print the count of such companies
-Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame

In [2]:
import os, sys
import urllib
import pyspark.sql.functions as F
import re
import unicodedata
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Window
from pyspark.sql.functions import col, size, split, when, udf, length, lower
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import row_number
from pyspark.sql.types import BooleanType

### Set Environment Variables for PySpark

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

### Initiate and configure Spark Session and Context

In [4]:
spark = SparkSession \
    .builder \
    .appName("Intro to Apache Spark") \
    .config("spark.cores.max", "4") \
    .config('spark.executor.memory', '8G') \
    .config('spark.driver.maxResultSize', '8g') \
    .config('spark.kryoserializer.buffer.max', '512m') \
    .config("spark.driver.cores", "4") \
    .getOrCreate()

sc = spark.sparkContext
print("Using Apache Spark Version", spark.version)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 18:14:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Apache Spark Version 4.0.1


### Load Crunchbase Dataset and Get Record Count

In [5]:
cb_sdf = spark.read.option("header", "true") \
                   .option("delimiter", ",") \
                   .option("inferSchema", "true") \
                   .csv("crunchbase_odm_orgs.csv") # use the just file name because the file is located in the same folder.
#cb_sdf.show()
cb_sdf.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- primary_role: string (nullable = true)
 |-- cb_url: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- homepage_url: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- facebook_url: string (nullable = true)
 |-- twitter_url: string (nullable = true)
 |-- linkedin_url: string (nullable = true)
 |-- combined_stock_symbols: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- short_description: string (nullable = true)



#### Display Column names of the Spark DataFrame

In [6]:
cb_sdf.columns


['uuid',
 'name',
 'type',
 'primary_role',
 'cb_url',
 'domain',
 'homepage_url',
 'logo_url',
 'facebook_url',
 'twitter_url',
 'linkedin_url',
 'combined_stock_symbols',
 'city',
 'region',
 'country_code',
 'short_description']

### Preview the first 10 rows of the Spark DataFrame

In [7]:
! pip install pandas
cb_sdf.limit(10).toPandas()

,uuid,name,type,primary_role,cb_url,domain,homepage_url,logo_url,facebook_url,twitter_url,linkedin_url,combined_stock_symbols,city,region,country_code,short_description
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,company,https://www.crunchbase.com/organization/wetpai...,wetpaint.com,http://www.wetpaint.com/,https://res.cloudinary.com/crunchbase-producti...,https://www.facebook.com/Wetpaint,https://twitter.com/wetpainttv,https://www.linkedin.com/company/wetpaint,None,New York,New York,USA,Wetpaint offers an online social publishing pl...
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,company,https://www.crunchbase.com/organization/zoho?u...,zoho.com,https://www.zoho.com/,https://res.cloudinary.com/crunchbase-producti...,http://www.facebook.com/zoho,http://twitter.com/zoho,http://www.linkedin.com/company/zoho-corporati...,None,Pleasanton,California,USA,"Zoho offers a suite of business, collaboration..."
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,company,https://www.crunchbase.com/organization/digg?u...,digg.com,http://www.digg.com,https://res.cloudinary.com/crunchbase-producti...,http://www.facebook.com/digg,http://twitter.com/digg,http://www.linkedin.com/company/digg,None,New York,New York,USA,Digg Inc. operates a website that enables its ...
3,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,company,https://www.crunchbase.com/organization/facebo...,facebook.com,http://www.facebook.com,https://res.cloudinary.com/crunchbase-producti...,https://www.facebook.com/facebook/,https://twitter.com/facebook,http://www.linkedin.com/company/facebook,nasdaq:FB,Menlo Park,California,USA,Facebook is an online social networking servic...
4,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,organization,investor,https://www.crunchbase.com/organization/accel?...,accel.com,http://www.accel.com,https://res.cloudinary.com/crunchbase-producti...,http://www.facebook.com/accel,http://twitter.com/accel,https://www.linkedin.com/company/accel-vc/,None,Palo Alto,California,USA,Accel is an early and growth-stage venture cap...
5,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,organization,company,https://www.crunchbase.com/organization/omnidr...,omnidrive.com,http://www.omnidrive.com,https://res.cloudinary.com/crunchbase-producti...,http://www.facebook.com/Nomadesk,http://twitter.com/Nomadesk,None,None,Palo Alto,California,USA,Omnidrive provides online storage facilities t...
6,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,organization,company,https://www.crunchbase.com/organization/geni?u...,geni.com,http://www.geni.com,https://res.cloudinary.com/crunchbase-producti...,https://www.facebook.com/genidotcom/,http://twitter.com/geni,https://www.linkedin.com/company/geni-com/about/,None,West Hollywood,California,USA,Geni is an online community of genealogists co...
7,180ebf67-68d0-2316-e93d-8e1e546330ba,Flektor,organization,company,https://www.crunchbase.com/organization/flekto...,flektor.com,http://www.flektor.com,https://res.cloudinary.com/crunchbase-producti...,None,None,None,None,Culver City,California,USA,Flektor is a mash-up platform that enables con...
8,d70777cc-14bd-2416-0692-5a483781b78b,Fox Interactive Media,organization,company,https://www.crunchbase.com/organization/fox-in...,fox.com,https://www.fox.com/,https://res.cloudinary.com/crunchbase-producti...,None,None,https://www.linkedin.com/company/fox-interacti...,None,Beverly Hills,California,USA,Fox Interactive Media is a network of companie...
9,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,company,https://www.crunchbase.com/organization/twitte...,twitter.com,http://www.twitter.com,https://res.cloudinary.com/crunchbase-producti...,https://www.facebook.com/twitterinc,http://twitter.com/twitter,http://www.linkedin.com/company/twitter,nyse:TWTR,San Francisco,California,USA,Twitter is a social networking platform that a...


In [8]:
from pyspark.sql.functions import col
cb_sdf.select(col("uuid"),col("name"),col("short_description")).show()

+--------------------+--------------------+--------------------+
|                uuid|                name|   short_description|
+--------------------+--------------------+--------------------+
|e1393508-30ea-8a3...|            Wetpaint|Wetpaint offers a...|
|bf4d7b0e-b34d-2fd...|                Zoho|Zoho offers a sui...|
|5f2b40b8-d1b3-d32...|                Digg|Digg Inc. operate...|
|df662812-7f97-0b4...|            Facebook|Facebook is an on...|
|b08efc27-da40-505...|               Accel|Accel is an early...|
|60485007-8856-bba...|           Omnidrive|Omnidrive provide...|
|4111dc8b-c0df-2d2...|                Geni|Geni is an online...|
|180ebf67-68d0-231...|             Flektor|Flektor is a mash...|
|d70777cc-14bd-241...|Fox Interactive M...|Fox Interactive M...|
|5da6106f-0d27-0d3...|             Twitter|Twitter is a soci...|
|3d16cb4c-911e-75c...|         StumbleUpon|StumbleUpon is a ...|
|56b5f0c1-855f-2eb...|              Gizmoz|Gizmoz develops p...|
|21e77067-5537-408...|   

#### **Select columns into RDD, format them into str, and back to dataframe**

In [9]:
cb_rdd = cb_sdf.select('*').rdd.map(lambda row: [str(row[i]) for i in ['uuid','name','domain']])
cb_sdf2 = spark.createDataFrame(cb_rdd,['uuid','name','domain'])
cb_sdf2.show(10)

+--------------------+--------------------+-------------+
|                uuid|                name|       domain|
+--------------------+--------------------+-------------+
|e1393508-30ea-8a3...|            Wetpaint| wetpaint.com|
|bf4d7b0e-b34d-2fd...|                Zoho|     zoho.com|
|5f2b40b8-d1b3-d32...|                Digg|     digg.com|
|df662812-7f97-0b4...|            Facebook| facebook.com|
|b08efc27-da40-505...|               Accel|    accel.com|
|60485007-8856-bba...|           Omnidrive|omnidrive.com|
|4111dc8b-c0df-2d2...|                Geni|     geni.com|
|180ebf67-68d0-231...|             Flektor|  flektor.com|
|d70777cc-14bd-241...|Fox Interactive M...|      fox.com|
|5da6106f-0d27-0d3...|             Twitter|  twitter.com|
+--------------------+--------------------+-------------+
only showing top 10 rows


### **Question 1. Find all companies with names that contain exactly three words**

In [10]:
from pyspark.sql.functions import size, split, col

cb_3words = cb_sdf.filter((col("primary_role") == "company")) \
                  .withColumn("word_count", size(split("name", " "))) \
                  .filter(col("word_count") == 3)

cb_3words.select("name", "city", "region", "country_code").show(10, truncate=False)

cb_3words.count()


+---------------------+-------------+------------+------------+
|name                 |city         |region      |country_code|
+---------------------+-------------+------------+------------+
|Fox Interactive Media|Beverly Hills|California  |USA         |
|Gannett Co Inc       |Mclean       |Virginia    |USA         |
|HOT or NOT           |San Francisco|California  |USA         |
|Funny Or Die         |San Mateo    |California  |USA         |
|Peak Steal Buildings |Morgan       |Georgia     |USA         |
|Kosiso - store       |Chicago      |Illinois    |USA         |
|The Accelerator Group|Mclean       |Virginia    |USA         |
|Real Time Matrix     |Oakland      |California  |USA         |
|Keep Highways Safe   |Jenkintown   |Pennsylvania|USA         |
|Red Light Center     |NULL         |NULL        |NULL        |
+---------------------+-------------+------------+------------+
only showing top 10 rows


204396

### **Question 2.Find all companies located in the state of New Jersey:**

In [11]:
cb_nj = cb_sdf.filter((col("region") == "New Jersey")&(col("primary_role") == "company"))
cb_nj.select("name","city", "region", "country_code").show()

# Print the count of such companies
cb_nj.count()

+-------------------+--------------+----------+------------+
|               name|          city|    region|country_code|
+-------------------+--------------+----------+------------+
|           Blogsigs|       Madison|New Jersey|         USA|
|              MSNBC|      Secaucus|New Jersey|         USA|
|            BitWine|       Tenafly|New Jersey|         USA|
|          GoAmerica|    Hackensack|New Jersey|         USA|
|          Phone.com|        Newark|New Jersey|         USA|
|           Phanfare|     Princeton|New Jersey|         USA|
|              Vidyo|    Hackensack|New Jersey|         USA|
|          eventsbot|    Piscataway|New Jersey|         USA|
|       CheerOutLoud|   Perth Amboy|New Jersey|         USA|
|        Jet Numbers|East Brunswick|New Jersey|         USA|
|       ShareMethods|  South Orange|New Jersey|         USA|
|      Winescorecard|  Lambertville|New Jersey|         USA|
|              ePrep|     Princeton|New Jersey|         USA|
|     Origin Digital|   

10215

### **Question3. Add a "TechDomain" column to the DataFrame where the row entries are set to 1 if the "domain" field contains "github.io", and 0 otherwise:**

In [12]:
from pyspark.sql.functions import when

cb_tech = (cb_sdf.filter(col("primary_role") == "company")\
                .withColumn("TechDomain", when(col("domain").contains("github.io"),1).otherwise(0))
          )

cb_tech_info = (cb_tech.filter(col("TechDomain") == 1)\
                      .select("name" , "city", "region", "country_code", "TechDomain")\
               )

cb_tech_info.show() # show the result
cb_tech_info.count() # show how many rows are in the dataframe.
                            

                            

+--------------------+-------------+------------+------------+----------+
|                name|         city|      region|country_code|TechDomain|
+--------------------+-------------+------------+------------+----------+
|               OSSEC|San Francisco|  California|         USA|         1|
|            Fosstrak|San Francisco|  California|         USA|         1|
|       littlesnapper|         NULL|        NULL|        NULL|         1|
|  Associação CRIArte|         NULL|        NULL|        NULL|         1|
|           Bros Labs|        Hanoi|NA - Vietnam|         VNM|         1|
|             Telcial|      Brenham|       Texas|         USA|         1|
|      Dev Free Casts|         NULL|        NULL|        NULL|         1|
|  Ojas Open Platform|San Francisco|  California|         USA|         1|
|       Bababa Studio|San Francisco|  California|         USA|         1|
|             Nova IO|San Francisco|  California|         USA|         1|
|Zest - Discover E...|San Francisco|  

81

### **Question 4. Find all companies whose names contain at least one repeating consecutive letter (e.g., "Google", "Twitter", "Massa Inc.") using a Spark UDF function.**

#### - Print the count of such companies
#### - Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame


#### 1) Case-sensitive alphabetic repetition – only A–Z/a–z letters, and case matters ("Aa" ≠ repeat).

In [13]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# (1) When it is case sensitive. 

def has_repeating_letters(name):
    if name:
        return bool(re.search(r"(.)\1", name))
    return False

repeat_udf = udf(has_repeating_letters, BooleanType())

cb_repeat = cb_sdf.filter(col("primary_role") == "company") \
                  .withColumn("has_repeat", repeat_udf(col("name"))) \
                  .filter(col("has_repeat") == True)

cb_repeat.select("name", "city", "region", "country_code").show(10, truncate=False)

cb_repeat.count()

+---------------------+-------------+-----------+------------+
|name                 |city         |region     |country_code|
+---------------------+-------------+-----------+------------+
|Digg                 |New York     |New York   |USA         |
|Facebook             |Menlo Park   |California |USA         |
|Twitter              |San Francisco|California |USA         |
|MeetMoi              |New York     |New York   |USA         |
|Joost                |New York     |New York   |USA         |
|AddThis              |Vienna       |Virginia   |USA         |
|Thoof                |Austin       |Texas      |USA         |
|Hearst Communications|New York     |New York   |USA         |
|Gannett Co Inc       |Mclean       |Virginia   |USA         |
|AllPeers             |Oxford       |Oxfordshire|GBR         |
+---------------------+-------------+-----------+------------+
only showing top 10 rows


270633

#### 2) Case-insensitive alphabetic repetition – only A–Z/a–z letters, ignoring case ("Aa" = repeat).

In [14]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType
import re

# (2) only alphabet repetition case insensitive
def repeat_alpha(name):
    if not name: return False
    return bool(re.search(r"([A-Za-z])\1", name, re.I))


#### 3) All-character repetition – any character (letters, numbers, symbols, spaces, etc.) counts.

In [15]:
# (3) all type of chatacters repetition..
def repeat_any(name):
    if not name: return False
    return bool(re.search(r"(.)\1", name, re.I))

udf_alpha = udf(repeat_alpha, BooleanType())
udf_any   = udf(repeat_any, BooleanType())

df_check = (
    cb_sdf.filter(col("primary_role") == "company")
          .withColumn("repeat_alpha", udf_alpha(col("name")))
          .withColumn("repeat_any", udf_any(col("name")))
)





#### 4) Comparison – Case-sensitive vs. Case-insensitive vs. All-character repetition

In [16]:
# Comparison – Case-sensitive vs. Case-insensitive vs. All-character repetition

count_alpha_caseSensitive = cb_repeat.count()  # Check repetition of alphabets (case-sensitive)
count_alpha_caseInsensitive = df_check.filter(col("repeat_alpha")).count()  # Check repetition of alphabets (case-insensitive)
count_any = df_check.filter(col("repeat_any")).count()  # Check repetition of any character type

print("=========================================================================================")
print("Count of all companies whose names contain at least one repeating consecutive character:")
print(f"- Alphabet repetition (case-sensitive): {count_alpha_caseSensitive:,}")
print(f"- Alphabet repetition (case-insensitive): {count_alpha_caseInsensitive:,}")
print(f"- Any character repetition: {count_any:,}")

Count of all companies whose names contain at least one repeating consecutive character:
- Alphabet repetition (case-sensitive): 270,633
- Alphabet repetition (case-insensitive): 271,819
- Any character repetition: 274,090
